In [ ]:
# This code, when augmented with Alex Hawkins-Hooker's code, will generate the bipartite_pickle.pickle network that
# is read in, in the Testing_Metrics file.

# The following code reads in a list of philosopher's names, checks them against the set of canonical philosophers
# from the DBpedia dataset, and adds the name as a node in the graph along with the boolean attribute "canonical"
names = nx.get_node_attributes(phil_graph, 'name')

In [ ]:
names = [x for y,x in names.items()]

In [ ]:
cleaned_names = set()
for name in names:
    list_name = name.lower().split(' ')
    cleaned_name = list_name[-1]
    for f in list_name[:-1]:
        cleaned_name = cleaned_name + " " + f[0] 
    cleaned_names.add(cleaned_name)

In [ ]:
# Populate edges
edge_list = []

with open('/afs/inf.ed.ac.uk/user/s16/s1681453/data_justin/stn_edge_list_attrs_wos.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader) # Skip the header
    for row in reader:
        edge_list.append(((row[0], row[1]), row[2:]))

In [ ]:
phil_citations = {}

for citation in edge_list:
    if citation[0][0] in phil_citations:
        phil_citations[citation[0][0]] = phil_citations[citation[0][0]] + 1
    else:
        phil_citations[citation[0][0]] = 1
    if citation[0][1] in phil_citations:
        phil_citations[citation[0][1]] = phil_citations[citation[0][1]] + 1
    else:
        phil_citations[citation[0][1]] = 1

In [ ]:
import re

# Tag the canonical philosophers
correction_dict = {}

citation_graph = nx.Graph()
count = 0
with open('/afs/inf.ed.ac.uk/user/s16/s1681453/data_justin/stn_node_list_wos.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader) # Skip the header
    for row in reader:
        count = count + 1
        if count % 1000 == 0:
            print(count)
        added = False
        row_split = row[0].split(" ")
        for x in cleaned_names:
            x_split = x.split(" ")
            if row[0] == x:
                citation_graph.add_node(row[0], canonical=True)
                correction_dict[row[0]] = x
                added = True
            elif row[0] in x and row_split[0] == x_split[0]:
                # Count occurrences of (split) x[0] + x[1] vs occurrences of (split) x[0] + not x[1] and if ratio 
                # is too low then don't correct
                occurrences_x = 0
                occurrences_not_x = 0
                x_pattern = re.compile(x_split[0])
                not_x_pattern = re.compile(x_split[0] + " ")
                if len(x_split) > 1:
                    x_pattern = re.compile(x_split[0] + " " + x_split[1])
                    not_x_pattern = re.compile(x_split[0] + " " + "[^" + x_split[1] + "]")
                for phil in phil_citations:
                    if x_pattern.match(phil):
                        occurrences_x = occurrences_x + phil_citations[phil]
                    if not_x_pattern.match(phil):
                        occurrences_not_x = occurrences_not_x + phil_citations[phil]
                if occurrences_x - occurrences_not_x > 0:
                    citation_graph.add_node(row[0], canonical=True)
                    correction_dict[row[0]] = x
                    added = True
        if not added:
            citation_graph.add_node(row[0], canonical=False)

In [ ]:
def dictValOrOriginal(dict, original):
    if original in dict:
        return dict[original]
    else:
        return original

for edge in edge_list:
    citation_graph.add_edge(dictValOrOriginal(correction_dict, edge[0][0]), 
                            dictValOrOriginal(correction_dict, edge[0][1]), 
                            year=edge[1][0], journal=edge[1][1], 
                            article_id=edge[1][2], language=edge[1][3], keywords=edge[1][4])

In [ ]:
nx.write_gpickle(citation_graph, '/afs/inf.ed.ac.uk/user/s16/s1681453/data_justin/citation_graph_pickle')
# Unpickle with nx.read_gpickle(path)